# ISystemtap: An Interactive Systemtap Notebook
<img src="https://sourceware.org/systemtap/smileytap.svg"  width="100em" height="100em" style="float: left; padding: 1em">
<img src="https://cdn.pixabay.com/photo/2014/04/02/10/55/plus-304947_1280.png"  width="30em" height="30em" style="float: left; padding-top: 3em">
<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/3/38/Jupyter_logo.svg/1200px-Jupyter_logo.svg.png"  width="85em" height="85em" style="padding: 1em">

## Introduction
Welcome the ISystemtap. We will first introduce the core concept of **cells** and **namespaces**.

* **cell** : A single code block. These come in various forms, specified by the **magic** first line `%%foo`. To see all the magic commands use the `%%help` **magic**. Only one cell can be executed at one time.
* **namespace** : A group of cells, joined under a common name. This name is specified as the optional first argument of the **magic** command

We'll start with the 2 most important ISystemtap cells. The `%%edit` cell which contains Systemtap script blocks and the `%%run` cell which combines and runs the blocks.
To get started execute (`Ctrl-Enter`) the *helloworld* cells and the *global* cells

In [ ]:
%%edit helloworld
probe oneshot{
  printf("Hello ISystemtap\nI am in the namespace %s\n", module_name())
}

In [ ]:
%%run helloworld

In [ ]:
%%edit
global s = "If a namespace is not specified I am in the"

In [ ]:
%%edit
probe oneshot{
  printf("%s %s namespace\n", s, module_name())
}

In [ ]:
%%run

## Basic Controls
Now we will investigate the basic controls of ISystemtap.

Run the *controls* namespace and try starting and pausing the entire namespace as well as individual probes. You can also reset globals back to their initial values

When you are done you can stop the namespace by clicking the Stop button or typing `I, I` into the cell input while in command mode

In [ ]:
%%edit controls
global t = 0
probe timer.s(1) {
  t++
  printf("I have counted up to %d\n", t)
}

probe end {
  printf("I have completed\n")
}

In [ ]:
%%run controls

## Globals
Now let's discuss the *Globals* tab in more detail and introduce a new cell type, the  `%%python` cell.

We'll demonstrate this using one of the Systemtap example scripts: alias_suffixes.stp

In [ ]:
%%edit globals
# Uses alias suffixes to track time intervals for a subset of kernel
# functions. Based on func_time_stats.stp.

global start, intervals

# We can apply a suffix to multiple probe points designated by one alias:
probe miscellany = syscall.{open,close,read,write} { }

probe miscellany {
      start[name, tid()] = gettimeofday_us()
}

# The 'return' suffix is passed on to each of the underlying probe points:
probe miscellany.return {
      t = gettimeofday_us()
      if([name, tid()] in start) old_t = start[name, tid()]
      if (old_t) intervals[name] <<< t - old_t
      delete start[name, tid()]
}

probe begin {
      printf("Collecting data... click Stop to end\n")
}

probe end {
      foreach (name in intervals) {
              printf("intervals for %s -- min:%dus avg:%dus max:%dus count:%d\n",
                     name, @min(intervals[name]), @avg(intervals[name]),
                     @max(intervals[name]), @count(intervals[name]))
              print(@hist_log(intervals[name]))
      }
}

In [ ]:
%%run globals -g --suppress-time-limits --suppress-handler-errors

As you can see the recorded globals are all visible under the *Globals* tab.

The `%%python` cell has access to all of these globals. I'll use them to generate a new plot using bqplot

In [ ]:
%%python globals
from bqplot import *
import bqplot.pyplot as plt
# We can access the globals
print(start)
print(intervals['write']['@avg'])

# The bar labels and corresponding heights
labels  = [syscall for syscall in intervals.keys()]
counts  = [stats['@count'] for stats in intervals.values()]

scale_x = OrdinalScale()
scale_y = LinearScale()
fig = plt.figure(
    title="Total Hits",
    title_style={"font-size": "2em", "fill": "red"},
)
def_tt = Tooltip(fields=['x', 'y'], labels = ['Syscall','Hits'])
bar = plt.bar(x = labels, y = counts,
                scales = {'x': scale_x, "y": scale_y},
                colors = ['red'], stroke = 'white', stroke_width=0.4,
                tooltip = def_tt)
# This is how we display plots using ISystemtap
display(fig)

## Examples and Probe Listing
Now we'll discuss the `%%examples` cell. This cell can search through, preview and run the Systemtap examples

Try out the `regex.stp` script with the argument 

functioncallcount.stp

In [ ]:
%%examples

We also have the `%%probes` cell. This cell can list availible probes and their context variables

Try listing `process.*`

In [ ]:
%%probes

## Writing our own scripts
Finally we'll discuss the `%%script` cell which is just a combination of the `%%edit` and `%%run` cells, Like `%%run` cells, `%%script` cells can take regular systemtap options, but `%%script`s can also take arguments (with the `--args=(a,b,c)` flag)

Let's try and write our own script, and take advantage of the **Systemtap Language Server** which allows for `tab` completion.

In [ ]:
%%script my_own_script -v -g --suppress-time-limits --suppress-handler-errors --args(10)
/*
* Write a script which prints "Hello World" at the start
* Then after 10 seconds, prints a histogram for the top 4 most used systemcalls
*/

# TODO: Define a global called word1
gl word1 = "Hello"
# ^

global word2 = "World"

probe begin {
  # TODO: Complete the second argument to word2
  printf("%s %s\n", word1, wo)
  #                          ^
}

global runtime = 1
# TODO: Complete the probe point to timer.s(1)
probe tim{
#        ^
  runtime++
  printf("%d seconds left\n", $1 - runtime)
  if(runtime >= $1){
    exit()
  }
}

global syscall_counts
# TODO: Complete the process path to "/bin/python"
probe process("/bin/pyth").syscall{
#                       ^
  if(execname() == "jupyter-lab"){
    # TODO: Complete the context variable to $syscall
    syscall_counts[syscall_name($sys)] <<< 1
    #                               ^
  }
}

probe end {
  # Iterate over the processeses
  foreach (syscall in syscall_counts- limit 4) {
    # TODO: Complete the macro to @hist_linear
    print(@(syscall_counts[syscall], 1, 1, 1))
    #      ^
  }
}

For more information on availible Systemtap functions, probes and tapsets take a look at the corresponding man pages. These can be accessed as follows

In [ ]:
!man function::module_name
!echo ------------------------------------------------------------------------------
!man tapset::dentry